In [1]:
import math as m

In [62]:
from scipy.stats import linregress
import numpy as np
import vice
output = vice.output("../outputs/angular-momentum-dilution/betaphiin0p7")
zone_width = 0.1
radii = []
sigmag = []
for i in range(30, 120):
    zone = output.zones["zone%d" % (i)]
    radii.append(zone_width * i)
    area = np.pi * ((radii[-1] + zone_width)**2 - radii[-1]**2)
    sigmag.append(zone.history["mgas"][-1] / area)
lnsigmag = [m.log(_) for _ in sigmag]
prof = linregress(radii, lnsigmag)
dlnsigmag_dr = prof.slope
print(-1 / dlnsigmag_dr)

5.395428156691953


Tilting of the Metallicity Gradient
==

GCE models with radial gas flows generally predict some level of tilting in the metallicity gradient over time, even on small levels.
In this regard, the fact that the level of tilting observed between mono-age populations is relatively minimal is physically interesting.
Even though there are arguments for variations between $\nabla \approx -0.15$ kpc$^{-1}$ and $\nabla \approx -0.05$ kpc$^{-1}$, this is still small in the sense that it's well within an order of magnitude.

By the symmetry of second derivatives:

$$
\begin{align}
\frac{\partial}{\partial t} \left(
\frac{\partial \ln Z_\alpha}{\partial R}\right)
&=
\frac{-1}{Z_\alpha^2} \frac{\partial Z_\alpha}{\partial t} \frac{\partial Z_\alpha}{\partial R} +
\frac{1}{Z_\alpha}\frac{\partial^2 Z_\alpha}{\partial t \partial R}
\\
&= \frac{1}{Z_\alpha} \left( \frac{\partial^2 Z_\alpha}{\partial R \partial t} -
\frac{\partial \ln Z_\alpha}{\partial R} \frac{\partial Z_\alpha}{\partial t} \right)
\\
&= \frac{1}{Z_\alpha} \left[ \frac{\partial}{\partial R} \left(
\frac{y_\alpha}{\tau_\star} -
\frac{Z_\alpha}{\tau_\star} \left(
1 + \eta - \mu_\alpha - r + \tau_\star \frac{\dot \Sigma_g}{\Sigma_g}
\right)
\right) -
\frac{\partial \ln Z_\alpha}{\partial R} \left(
\frac{y_\alpha}{ \tau_\star} -
\frac{Z_\alpha}{\tau_\star} \left(
1 + \eta - \mu_\alpha - r +
\tau_\star \frac{\dot \Sigma_g}{\Sigma_g}
\right)
\right)
\right].
\end{align}
$$

At this point, for compactness, it is helpful to substitute in the following relation:

$$
\begin{align}
Z_{\alpha,\text{eq}}
&= \frac{y_\alpha}{1 + \eta - \mu_\alpha - r + \tau_\star \dot \Sigma_g / \Sigma_g}
\\
\implies \frac{y_\alpha}{Z_{\alpha,\text{eq}}}
&= 1 + \eta - \mu_\alpha - r + \tau_\star \dot \Sigma_g / \Sigma_g,
\end{align}
$$

which leads to the following next step:

$$
\begin{align}
\implies \frac{\partial}{\partial t} \left(
\frac{\partial \ln Z_\alpha}{\partial R} \right)
&= \frac{1}{Z_\alpha} \left[
\frac{\partial}{\partial R} \left(
\frac{y_\alpha}{\tau_\star} -
\frac{y_\alpha}{\tau_\star} \frac{Z_\alpha}{Z_{\alpha,\text{eq}}}
\right) -
\frac{\partial \ln Z_\alpha}{\partial R} \left(
\frac{y_\alpha}{\tau_\star} -
\frac{y_\alpha}{\tau_\star} \frac{Z_\alpha}{Z_{\alpha,\text{eq}}}
\right)
\right]
\\
&= \frac{y_\alpha}{Z_\alpha} \left[
\frac{\partial}{\partial R} \left(
\frac{1}{\tau_\star} \left(
1 - \frac{Z_\alpha}{Z_{\alpha,\text{eq}}}
\right)
\right)
- \frac{\partial \ln Z_\alpha}{\partial R} \left(
\frac{1}{\tau_\star} \left(
1 - \frac{Z_\alpha}{Z_{\alpha,\text{eq}}}
\right)
\right)
\right]
\\
&= \frac{y_\alpha}{Z_\alpha} \left[
\frac{-1}{\tau_\star} \frac{\partial \ln \tau_\star}{\partial R}
\left(1 - \frac{Z_\alpha}{Z_{\alpha,\text{eq}}} \right) -
\frac{Z_\alpha}{\tau_\star Z_{\alpha,\text{eq}}} \left(
\frac{\partial \ln Z_\alpha}{\partial R} -
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
\right) -
\frac{1}{\tau_\star} \frac{\partial \ln Z_\alpha}{\partial R} +
\frac{Z_\alpha}{\tau_\star Z_{\alpha,\text{eq}}} \frac{\partial \ln Z_\alpha}{\partial R}
\right]
\\
&= \frac{y_\alpha}{Z_\alpha} \left[
\frac{-1}{\tau_\star} \frac{\partial \ln \tau_\star}{\partial R}
\left(1 - \frac{Z_\alpha}{Z_{\alpha,\text{eq}}} \right) +
\frac{Z_\alpha}{\tau_\star Z_{\alpha,\text{eq}}}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} -
\frac{1}{\tau_\star} \frac{\partial \ln Z_\alpha}{\partial R}
\right]
\end{align}
$$

The above is a compact way to write the expression for the tilt rate.
We are interested in finding that gradient $\nabla$ such that $\dot \nabla = 0$, which is the equilibrium gradient $\nabla_\text{eq}$.
From the above, it follows that $\dot \nabla \ln Z_\alpha = 0$ when

$$
\begin{align}
\frac{\partial \ln Z_\alpha}{\partial R}
&= \frac{Z_\alpha}{Z_{\alpha,\text{eq}}} \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} -
\left(1 - \frac{Z_\alpha}{Z_{\alpha,\text{eq}}}\right)
\frac{\partial \ln \tau_\star}{\partial R}
\end{align}
$$

With a Kennicutt-Schmidt relation of power-law index $N$:

$$
\begin{align}
\frac{\partial \ln \tau_\star}{\partial R} &=
\left(1 - N\right) \frac{\partial \ln \Sigma_g}{\partial R}
\\
\implies \nabla_\text{eq}
&= \frac{1}{\ln 10} \left[
\frac{Z_\alpha}{Z_{\alpha,\text{eq}}}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} +
\left(1 - \frac{Z_\alpha}{Z_{\alpha,\text{eq}}}\right)
(N - 1) \frac{\partial \ln \Sigma_g}{\partial R}
\right].
\end{align}
$$

Based on the above expression, we can say that metallicity gradients should generically experience two regimes, given sufficient time.
The first regime occurs in the early epochs of disk formation when the local metallicity $Z_\alpha \ll Z_{\alpha,\text{eq}}$.
In this limit,

$$
\begin{align}
\nabla_\text{eq} &\approx \frac{N - 1}{\ln 10} \frac{\partial \ln \Sigma_g}{\partial R}
\\
&\lesssim \frac{-0.5}{R_g \ln 10}
\\
&\lesssim -0.058\text{ kpc}^{-1},
\end{align}
$$

where we denote this approximation as an upper limit, since the surface density of the ISM likely followed a sharper density profile at early epochs (i.e., smaller $R_g$).
In this case, the exponential scaling of $Z_\alpha$ follows a scale radius that is about twice that of the ISM surface density (i.e., $R_\alpha \approx 2 R_g$).

The second regime is $Z_\alpha \rightarrow Z_{\alpha,\text{eq}}$, which occurs at late epochs; in some chemical evolution scenarios, on timescales much longer than the disk lifetime.
In the equilibrium scenario, it happens in the first $\sim$few Gyr of the thin disk epoch.
In this regime,

$$
\nabla_\text{eq} \approx \frac{1}{\ln 10} \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}.
$$

The radial profile is dominated by the equilibrium state.

Whether or not the gradient steepens, shallows, or holds steady on long timescales is a model-dependent question.
It depends on how star formation efficiency varied with radius at early epochs and what the eventual equilibrium gradient is.
If there are no substantial variations in star formation efficiency with radius at early times, then $\nabla_\text{eq} \approx 0$ at early times, and a relatively flat metallicity gradient is expected.
If $\nabla \ln Z_{\alpha,\text{eq}} \ll 0$, then the gradient should steepen significantly over the course of the disk lifetime.

Analytic Solutions to the Equilibrium Metallicity Gradient
==

This notebook details analytic solutions to the equilibrium metallicity gradient for any choice of SFE timescale, radial gas velocity, outflow prescription, etc.
We start with the analytic solution to the equilibrium alpha element abundance in a galaxy whose late-time SFH declines exponentially with a timescale $\tau_\text{sfh}$:

$$
Z_{\alpha,\text{eq}} = \frac{y_\alpha}{1 + \eta - \mu_\alpha - r - \tau_\star / N \tau_\text{sfh}},
$$

where $y_\alpha$ is the fractional net yield of alpha elements (solar masses of new metals produced per solar mass of star formation).
$\eta$ is the mass loading factor describing the outflow efficiency ($\eta \equiv \dot \Sigma_\star / \dot \Sigma_\text{out}$).
$\mu_\alpha$ is the flow coefficient $\mu_\alpha = \dot \Sigma_{\text{flow},\alpha} / \dot \Sigma_\star$, given by

$$
\mu_\alpha \equiv -\tau_\star v_{r,g} \left[
\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R} +
\frac{\partial \ln Z_{\alpha}}{\partial R}
\right].
$$

$r$ is the recycling fraction $r = \dot{M}_r / \dot{M}_\star \approx 0.4$ for a Kroupa (2001) IMF.
$\tau_\star \equiv \Sigma_g / \dot \Sigma_\star$ is the star formation efficiency timescale.
$N$ is the power-law index of the Kennicutt-Schmidt relation.
The gas flow coefficient is given by:

$$
\mu_g \equiv -\tau_\star v_{r,g} \left[
\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right].
$$

The simplest expression for the gradient in $Z_{\alpha,\text{eq}}$ comes from making the following substitution:

$$
\begin{align}
Z_{\alpha,\text{eq}}
&= \frac{y_\alpha}{
1 + \eta - \mu_g - r - \tau_\star / N \tau_\text{sfh} +
\tau_\star v_{r,g} \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
}
\\
&= \frac{y_\alpha}{
\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} +
\tau_\star v_{r,g} \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
}
\\
&\approx y_\alpha \frac{\dot \Sigma_\star}{\dot \Sigma_\text{in}}
\\
\implies \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
&\approx \frac{-\partial \ln (\dot \Sigma_\text{in} / \dot \Sigma_\star)}{\partial R}
\\
&= \frac{1}{1 + \eta - \mu_g - r - \tau_\star / N \tau_\text{sfh}} \left[
\frac{\partial \mu_g}{\partial R} +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right) -
\frac{\partial \eta}{\partial R}
\right]
\end{align}
$$

This approximation is valid as long as $\partial \ln Z_{\alpha,\text{eq}} / \partial R \ll \dot \Sigma_\text{in} / \dot \Sigma_\star$.
Our numerical models indicate that this condition is met across the disk in all models except the River limit, which has $\dot \Sigma_\text{in} \rightarrow 0$ by definition.
In detail, this term is responsible for the exponential cutoff in the radial metallicity profile seen at large $R$; the gradient becomes exponentially steeper with radius once this term dominates $\mu_\alpha$.
For realistic gradients, the slope is more uniform across much of the disk and significantly shallower than the gradients that arise in this limit.

This expression for the equilibrium gradient further reduces for a given assumption about the radial flow velocities, which sets the form of $\mu_g$; of course, assumptions regarding other GCE parameters also simplify the expression.

No Radial Flow
--

With no radial flows, the equilibrium gradient simplifies greatly:

$$
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} =
\frac{\tau_\star / N \tau_\text{sfh}}{1 - r - \tau_\star / N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right).
$$

The only substitution worth making is for $\nabla \ln \tau_\star$:

$$
\begin{align}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} &=
\frac{\tau_\star / N \tau_\text{sfh}}{1 - r - \tau_\star / N \tau_\text{sfh}} \left[
(1 - N) \frac{\partial \ln \Sigma_g}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right]
\\
&= \frac{\tau_\star / N \tau_\text{sfh}}{1 - r - \tau_\star / N \tau_\text{sfh}} \left(
\frac{N - 1}{R_g} - \frac{1}{R_\text{sfh}} \right)
\end{align}
$$

In [42]:
taustar = 6 # Gyr
tausfh = 13 # Gyr
N = 1.5
recycling = 0.4
Rg = 3.75 # kpc
Rsfh = 4.7 #kpc
z_on_zeq = 10**-0.1

numerator = taustar / (N * tausfh)
denominator = 1 - recycling - numerator
dlnzeq_dr = numerator / denominator * ((N - 1) / Rg - 1 / Rsfh)
print(dlnzeq_dr / m.log(10))
gradeq = z_on_zeq * dlnzeq_dr + (1 - z_on_zeq) * (1 - N) / Rg
gradeq /= m.log(10)
print(gradeq)

-0.03631278982692364
-0.04075388927347348


Constant Velocity
--

If the velocity profile is flat with radius, then by definition:

$$
\begin{align}
\frac{\partial \ln v_{r,g}}{\partial R} &= 0
\\
\implies \mu_g &= -\tau_\star v_{r,g} \left(
\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} \right)
\\
&= \tau_\star v_{r,g} \left(\frac{1}{R_g} - \frac{1}{R}\right),
\end{align}
$$

where $R_g \approx 3.75$ kpc is the scale radius of the ISM surface density.
Continuing:

$$
\begin{align}
\frac{\partial \mu_g}{\partial R}
&= \frac{-\partial \tau_\star}{\partial R} v_{r,g} \left(
\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} \right) -
\tau_\star v_{r,g} \left(
\frac{-1}{R^2} + \frac{\partial^2 \ln \Sigma_g}{\partial R^2}\right)
\\
&\approx -\tau_\star v_{r,g} \left[
\frac{\partial \ln \tau_\star}{\partial R} \left(
\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} \right) -
\frac{1}{R^2}
\right]
\\
&\approx -\tau_\star v_{r,g} \left[
(1 - N)\frac{\partial \ln \Sigma_g}{\partial R} \left(
\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} \right) -
\frac{1}{R^2}
\right]
\\
&\approx \tau_\star v_{r,g} \left(
\frac{1 - N}{R R_g} -
\frac{1 - N}{R_g^2} +
\frac{1}{R^2}
\right)
\end{align}
$$

Plugging into the expression for $\partial \ln Z_{\alpha,\text{eq}} / \partial R$ and applying the limit that $\eta \rightarrow 0$:

$$
\begin{split}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
&= \left[
\tau_\star v_{r,g} \left(
\frac{1 - N}{R R_g} -
\frac{1 - N}{R_g^2} +
\frac{1}{R^2}
\right) +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
(1 - N)\frac{\partial \ln \Sigma_g}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right)
\right]
\left(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} -
\tau_\star v_{r,g} \left(\frac{1}{R_g} - \frac{1}{R}\right)
\right)^{-1}
\\
&= \left[
\tau_\star v_{r,g} \left(
\frac{1 - N}{R R_g} -
\frac{1 - N}{R_g^2} +
\frac{1}{R^2}
\right) +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{N - 1}{R_g} - \frac{1}{R_\text{sfh}}
\right)
\right]
\left(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} -
\tau_\star v_{r,g} \left(\frac{1}{R_g} - \frac{1}{R}\right)
\right)^{-1}
\\
&\approx \left[
\tau_\star v_{r,g} \frac{N - 1}{R_g^2} +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{N - 1}{R_g} - \frac{1}{R_\text{sfh}}
\right)
\right]
\left(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} -
\frac{\tau_\star v_{r,g}}{R_g}
\right)^{-1},
\end{split}
$$

where we approximate $1 / R \rightarrow 0$ in the final line.

In [36]:
N = 1.5 # K-S power-law index
Rg = 3.75 # kpc
vgas = -1.5 # km/s ~ kpc/Gyr
taustar = 6 # Gyr
recycling = 0.4
tausfh = 13 # Gyr
Rsfh = 4.7 # kpc
z_on_zeq = 10**-0.1
# R = 8 # kpc

numerator = taustar / (N * tausfh) * ((N - 1) / Rg - 1 / Rsfh)
numerator += taustar * vgas * (N - 1) / Rg**2
# numerator += taustar * vgas * ((1 - N) / (R * Rg) - (1 - N) / Rg**2 + 1 / R**2)
denominator = 1 - recycling - taustar / (N * tausfh)
denominator -= taustar * vgas / Rg
dlnzeqdr = numerator / denominator
print(dlnzeqdr / m.log(10))
gradeq = z_on_zeq * dlnzeqdr + (1 - z_on_zeq) * (1 - N) / Rg
gradeq /= m.log(10)
print(gradeq)

-0.05556153274456677
-0.056043709255906474


We therefore estimate $\nabla_\text{eq} [\alpha/\text{H}] = -0.053$ kpc$^{-1}$, which is in reasonable agreement with our numerical models ($\nabla [\alpha/\text{H}] \approx -0.06$ kpc$^{-1}$) for an analytic approximation.

Potential Well Deepening
--

In the potential well deepening scenario, the velocity profile is linear with a slope set by the growth in stellar mass:

$$
\begin{split}
v_{r,g} &= -R \gamma \frac{\partial \ln M_\star}{\partial t}
\\
\implies \frac{\partial \ln v_{r,g}}{\partial R} &= \frac{1}{R}
\\
\implies \mu_g &= -\tau_\star v_{r,g} \left(
\frac{2}{R} + \frac{\partial \ln \Sigma_g}{\partial R} \right)
\\
\implies \frac{\partial \mu_g}{\partial R}
&= -\tau_\star v_{r,g} \left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)\left(
\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} \right) -
\tau_\star v_{r,g} \left(
\frac{-2}{R^2} + \frac{\partial^2 \ln \Sigma_g}{\partial R^2} \right)
\\
&= -\tau_\star v_{r,g} \left[
\left(\frac{1}{R} + (1 - N) \frac{\partial \ln \Sigma_g}{\partial R}\right)
\left(\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R}\right) -
\frac{2}{R^2} +
\frac{\partial^2 \ln \Sigma_g}{\partial R^2}
\right]
\\
&\approx \tau_\star v_{r,g} \frac{N - 1}{R_g^2},
\end{split}
$$

where we approximate $1 / R \rightarrow 0$ in the final line and substitute in $\partial \ln \Sigma_g / \partial R = -1 / R_g$.
We also approximate the gradient as being "locally exponential," in the sense that $\partial^2 \ln \Sigma_g / \partial R^2 \approx 0$.

This leads to the following approximation for $\partial \ln Z_{\alpha,\text{eq}} / \partial R$:

$$
\begin{split}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
&\approx \left[
\tau_\star v_{r,g} \frac{N - 1}{R_g^2} +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
(1 - N) \frac{\partial \ln \Sigma_g}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right)
\right] \left(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\tau_\star v_{r,g} \left(\frac{2}{R} +
\frac{\partial \ln \Sigma_g}{\partial R}\right)
\right)^{-1}
\\
&\approx \left[
\tau_\star v_{r,g} \frac{N - 1}{R_g^2} +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{N - 1}{R_g} -
\frac{1}{R_\text{sfh}}
\right)
\right] \left(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} -
\frac{\tau_\star v_{r,g}}{R_g}
\right)^{-1},
\end{split}
$$

which has the same form as our approximation for the constant velocity scenario.
The predictions are different, however, because the velocity and $Z_\alpha / Z_{\alpha,\text{eq}}$ are different.

In [19]:
N = 1.5 # K-S power-law index
Rg = 3.75 # kpc
# vgas = -0.075 # km/s ~ kpc/Gyr
R = 8 # kpc
gamma = 0.3
dlnmstar_dt = 2.0e9 / 5.0e10
vgas = -R * gamma * dlnmstar_dt
taustar = 6 # Gyr
recycling = 0.4 # unitless
tausfh = 13 # Gyr
Rsfh = 4.7 # kpc
z_on_zeq = 10**-0.2

numerator = taustar / (N * tausfh) * ((N - 1) / Rg - 1 / Rsfh)
numerator += taustar * vgas * (N - 1) / Rg**2
denominator = 1 - recycling - taustar / (N * tausfh)
denominator -= taustar * vgas / Rg
dlnzeqdr = numerator / denominator
print(dlnzeqdr / m.log(10))
gradeq = z_on_zeq * dlnzeqdr + (1 - z_on_zeq) * (1 - N) / Rg
gradeq /= m.log(10)
print(gradeq)

-0.043750890871555266
-0.04897470444018254


We therefore estimate $\nabla_\text{eq} [\alpha/\text{H}] \approx -0.048$ kpc$^{-1}$ for the PWD scenario with $\gamma = 0.2$, which is also in excellent agreement with our numerical models -- it is more or less halfway between the numerically predicted equilibrium and ISM gradients.

Angular Momentum Dilution
--

The AMD scenario is the most mathematically complex, but a straightforward solution can be found with a little sleight of hand.
The radial flow velocity is given by

$$
v_{r,g} = -R \frac{\dot \Sigma_\text{in}}{\Sigma_g} \left(1 - \beta_{\phi,\text{in}}\right),
$$

which allows one to multiply through by $\tau_\star$ and solve directly for $\dot \Sigma_\text{in} / \dot \Sigma_\star$:

$$
\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} =
\frac{-\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})}.
$$

The equilibrium abundance scales as the ratio of star formation per unit accretion, so we can solve directly for $\nabla \ln Z_{\alpha,\text{eq}}$ from here:

$$
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} = \frac{1}{R} -
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln v_{r,g}}{\partial R}
$$

The ratio $\dot \Sigma_\text{in} / \dot \Sigma_\star$ can also generally be expressed as:

$$
\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} = 1 + \eta - \mu_g - r - \tau_\star / N \tau_\text{sfh},
$$

which allows us to isolate the velocity derivative buried inside of $\mu_g$.
Also applying the limit that $\eta \rightarrow 0$:

$$
\begin{align}
1 - \mu_g - r - \tau_\star / N \tau_\text{sfh} &= \frac{-\tau_\star v_{r,g}}{R (1 - \beta_{phi,\text{in}})}
\\
\implies \tau_\star v_{r,g} \left(\frac{1}{R} +
\frac{\partial \ln \Sigma_g}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R} \right) &=
\frac{-\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})} -
\left(1 - r - \tau_\star / N \tau_\text{sfh}\right)
\\
\implies \frac{\partial \ln v_{r,g}}{\partial R} &=
\frac{-1}{R}\left(1 + \frac{1}{1 - \beta_{\phi,\text{in}}}\right) -
\frac{\partial \ln \Sigma_g}{\partial R} -
\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{\tau_\star v_{r,g}}
\end{align}
$$

Plugging everything in and combining terms results in the following expression for the gradient in the equilibrium abundance:

$$
\begin{align}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
&= \frac{1}{R} \left(2 + \frac{1}{1 - \beta_{\phi,\text{in}}}\right) +
N \frac{\partial \ln \Sigma_g}{\partial R} +
\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{\tau_\star v_{r,g}}
\\
&\approx \frac{1}{R (1 - \beta_{\phi,\text{in}})} +
\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{\tau_\star v_{r,g}} -
\frac{N}{R_g}
\end{align}
$$

In [53]:
R = 8 # kpc
betaphiin = 0.7 # unitless
Rg = 3.75 # kpc
taustar = 6 # Gyr
tausfh = 13 # Gyr
N = 1.5
vgas = -0.24 # km/s ~ kpc/Gyr
recycling = 0.4

# dlnv_dr = 0
# # dlnv_dr += 1 / Rg
# dlnv_dr -= 1 / R * (betaphiin - 2) / (betaphiin - 1)
# dlnv_dr -= (1 - recycling - taustar / (N * tausfh)) / (taustar * vgas)
# print(dlnv_dr)
# print(1 / 6)

dlnv_dr = 0
dlnv_dr -= 1 / R * (1 + 1 / (1 - betaphiin))
dlnv_dr += 1 / Rg
dlnv_dr -= (1 - recycling - taustar / (N * tausfh)) / (taustar * vgas)
print(dlnv_dr)
print(1 / 6)

-0.072008547008547
0.16666666666666666


In [63]:
R = 8 # kpc
betaphiin = 0.7
N = 1.5 # K-S power-law index
Rg = 3.75 # kpc
vgas = -0.24 # km/s ~ kpc/Gyr
taustar = 6 # Gyr
recycling = 0.4
tausfh = 13 # Gyr
Rsfh = 4.7 # kpc
z_on_zeq = 10**-0.2

print(1 / 6)
dlnvdr = -(1 - recycling - taustar / (N * tausfh)) / (taustar * vgas)
print(dlnvdr)
dlnvdr -= 1 / R * (betaphiin - 2) / (betaphiin - 1)
print(dlnvdr)
dlnvdr += 1 / Rg
print(dlnvdr)

0.16666666666666666
0.20299145299145296
-0.3386752136752137
-0.15685703185703187


In [48]:
R = 8 # kpc
betaphiin = 0.7
N = 1.5 # K-S power-law index
Rg = 3.75 # kpc
vgas = -0.24 # km/s ~ kpc/Gyr
taustar = 6 # Gyr
recycling = 0.4
tausfh = 13 # Gyr
Rsfh = 4.7 # kpc
z_on_zeq = 10**-0.2

dlnzeqdr = 1 / R
dlnzeqdr -= (N - 1) / Rg
dlnzeqdr -= 1 / 6
print(dlnzeqdr)

# dlnzeqdr = (1 - recycling - taustar / (N * tausfh)) / (taustar * vgas)
# dlnzeqdr -= (N + 1) / Rg
# dlnzeqdr += 1 / R * (2 * betaphiin - 3) / (betaphiin - 1)
# # dlnzeqdr += 1 / (R * (1 - betaphiin))
# # dlnzeqdr += 2 / R
# print(dlnzeqdr)
# print(dlnzeqdr / m.log(10))
# gradeq = z_on_zeq * dlnzeqdr + (1 - z_on_zeq) * (1 - N) / Rg
# gradeq /= m.log(10)
# print(gradeq)

-0.175


The equilibrium gradient, by definition:

$$
\begin{align}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} &=
\frac{1}{1 + \eta - \mu_\alpha - r - \tau_\star / N \tau_\text{sfh}}
\left[
\frac{\partial \mu_\alpha}{\partial R} +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right) -
\frac{\partial \eta}{\partial R}
\right]
\end{align}
$$

For the purposes of this notebook, it is notationally convenient to define

$$
\begin{align}
\gamma_\alpha &\equiv \frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R} + \frac{\partial \ln Z_{\alpha}}{\partial R}
\\
\gamma_g &\equiv \frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R},
\end{align}
$$

such that $\mu_g \equiv -\tau_\star v_{r,g} \gamma_g$ and $\mu_\alpha \equiv -\tau_\star v_{r,g} \gamma_\alpha$.

One of the first things we need is a solution to $\partial \mu_\alpha / \partial R$. By definition:

$$
\frac{\partial \mu_\alpha}{\partial R}
= \frac{\partial \mu_g}{\partial R} -
\tau_\star v_{r,g} \frac{\partial \ln Z_\alpha}{\partial R} \left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right) -
\tau_\star v_{r,g} \frac{\partial^2 \ln Z_\alpha}{\partial R^2}.
$$

For the purposes of this analytic solution, we approximate $\partial^2 \ln Z_\alpha / \partial R^2 \approx 0$. Therefore

$$
\frac{\partial \mu_\alpha}{\partial R}
\approx \frac{\partial \mu_g}{\partial R} -
\left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)
\tau_\star v_{r,g} \frac{\partial \ln Z_\alpha}{\partial R}.
$$

We now arrive at the following next step in the solution to the equilibrium gradient:

$$
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} \left(
1 + \eta - \mu_\alpha - r - \tau_\star / N \tau_\text{sfh}
\right)
\approx 
\frac{\partial \mu_g}{\partial R} -
\left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)
\tau_\star v_{r,g} \frac{\partial \ln Z_\alpha}{\partial R}
+
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right) -
\frac{\partial \eta}{\partial R}.
$$

After cross-multiplying the denominator to the left-hand side, the right hand side in the above expression depends only linearly on the metallicity gradient $\nabla \ln Z_\alpha$.
Each of the other terms have to do with the gas and do not explicitly depend on the metal abundance in this framework.

Zooming in on the left-hand side of the above relation:

$$
\begin{split}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} \left(
1 + \eta - \mu_\alpha - r - \tau_\star / N \tau_\text{sfh} \right)
&= \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} \left(
1 + \eta - \mu_g - r - \tau_\star / N \tau_\text{sfh} \right) -
\left(\mu_\alpha - \mu_g\right)
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
\\
&= \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} \left(
1 + \eta - \mu_g - r - \tau_\star / N \tau_\text{sfh} \right) +
\tau_\star v_{r,g} \left(
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}\right)^2
\end{split}
$$

At this point, it is helpful for compactness to substitute in the ratio of accretion per unit star formation.
From surface density conservation:

$$
\begin{align}
\dot \Sigma_g
&= \dot \Sigma_\text{in} -
\dot \Sigma_\star \left(1 + \eta - \mu_g - r\right)
\\
\implies \dot \Sigma_\text{in}
&= \dot \Sigma_\star \left(1 + \eta - \mu_g - r\right) +
\dot \Sigma_g
\\
\implies \frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star}
&= 1 + \eta - \mu_g - r + \tau_\star \frac{\dot \Sigma_g}{\Sigma_g}
\\
&\rightarrow 1 + \eta - \mu_g - r - \frac{\tau_\star}{N \tau_\text{sfh}} \text{ as } t \rightarrow \infty
\end{align}
$$

$\dot \Sigma_g / \Sigma_g = -1 / N \tau_\text{sfh}$ for an exponential SFH $\dot \Sigma_\star / \Sigma_\star = -1 / \tau_\text{sfh}$, where $N$ is the power-law index of the Kennicutt-Schmidt relation $\dot \Sigma_\star \propto \Sigma_g^N$ (i.e. $\partial \ln \dot \Sigma_\star = N \partial \ln \Sigma_g$).

Plugging in:

$$
\begin{align}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} \left(
1 + \eta - \mu_\alpha - r - \tau_\star / N \tau_\text{sfh} \right)
&= \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} \left(
1 + \eta - \mu_g - r - \tau_\star / N \tau_\text{sfh} \right) +
\tau_\star v_{r,g} \left(
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}\right)^2
\\
&= \frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} +
\tau_\star v_{r,g} \left(
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}\right)^2
\end{align}
$$

Plugging back in to the original expression and combining terms, we arrive at the following relation:

$$
\begin{align}
\tau_\star v_{r,g} \left(
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}\right)^2 +
\left(\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} +
\tau_\star v_{r,g} \left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)
\right) \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
&= \frac{\partial \mu_g}{\partial R} +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}\right) -
\frac{\partial \eta}{\partial R}
\\
\implies \tau_\star v_{r,g} \left(
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}\right)^2 +
\left(\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} +
\tau_\star v_{r,g} \left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)
\right) \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
&- \frac{\partial \mu_g}{\partial R} -
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}\right) +
\frac{\partial \eta}{\partial R} = 0
\end{align}
$$

The solution to the above is a complex quadratic, which is not necessarily unfeasible to implement.
However, for the purposes of an analytic solution, we neglect the squared gradient term: $(\partial \ln Z_{\alpha,\text{eq}} / \partial R)^2 \approx 0$.
This simplification results in the following linear approximation:

$$
\begin{align}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} &= \left[
\frac{\partial \mu_g}{\partial R} +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right) -
\frac{\partial \eta}{\partial R}
\right] \left(
\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} +
\tau_\star v_{r,g} \left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)
\right)^{-1}
\\
&= \left[
\frac{\partial \mu_g}{\partial R} +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right) -
\frac{\partial \eta}{\partial R}
\right] \left(
1 + \eta - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\tau_\star v_{r,g} \left(
\frac{1}{R} +
\frac{\partial \ln \Sigma_g}{\partial R} +
2\frac{\partial \ln v_{r,g}}{\partial R} +
\frac{\partial \ln \tau_\star}{\partial R}
\right)
\right)^{-1}
\end{align}
$$

This is our solution in its most general form. From here, some substitutions that are specific to this notebook.

Our power-law Kennicutt-Schmidt relation results in a simple relation between the derivatives in $\tau_\star$ and $\Sigma_g$:

$$
\begin{align}
\dot \Sigma_\star \equiv \Sigma_g \tau_\star^{-1} &\propto \Sigma_g^N
\\
\implies \tau_\star &\propto \Sigma_g^{1 - N}
\\
\implies \frac{\partial \ln \tau_\star}{\partial R} &= (1 - N)\frac{\partial \ln \Sigma_g}{\partial R}
\end{align}
$$

We also work in the $\eta = 0$ in this project. Substituting in these assumptions:

$$
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
= \left[
\frac{\partial \mu_g}{\partial R} +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
(1 - N)\frac{\partial \ln \Sigma_g}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right)
\right] \left(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\tau_\star v_{r,g} \left(
\frac{1}{R} +
(2 - N)\frac{\partial \ln \Sigma_g}{\partial R} +
2\frac{\partial \ln v_{r,g}}{\partial R}
\right)
\right)^{-1}
$$

Constant Velocity
--

In the limit of a flat velocity profile $v_{r,g} = $ constant,

$$
\frac{\partial \ln v_{r,g}}{\partial R} = 0,
$$

which leads to a simplified expression for $\mu_g$:

$$
\begin{align}
\mu_g &\rightarrow -\tau_\star v_{r,g} \left(
\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R}
\right)
\\
\implies \frac{\partial \mu_g}{\partial R}
&= \frac{-\partial \tau_\star}{\partial R} v_{r,g} \left(
\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R}\right) -
\tau_\star v_{r,g} \left(
\frac{-1}{R^2} +
\frac{\partial^2 \ln \Sigma_g}{\partial R^2}
\right)
\\
&\approx -\tau_\star v_{r,g} \left[
\frac{\partial \ln \tau_\star}{\partial R} \left(
\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} \right) -
\frac{1}{R^2}
\right]
\\
&= -\tau_\star v_{r,g} \left[
\frac{1 - N}{R} \frac{\partial \ln \Sigma_g}{\partial R} +
(1 - N) \left(\frac{\partial \ln \Sigma_g}{\partial R}\right)^2 -
\frac{1}{R^2}
\right]
\\
&= \tau_\star v_{r,g} \left[
\frac{1 - N}{R R_g} - \frac{1 - N}{R_g^2} + \frac{1}{R^2}
\right]
% \implies \frac{\partial \mu_g}{\partial R} &=
% \mu_g \frac{\partial \ln \tau_\star}{\partial R} -
% \tau_\star v_{r,g} \frac{\partial \gamma_g}{\partial R}
% \\
% &= -\tau_\star v_{r,g} \left[\left(
% \frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R}
% \right) (1 - N) \frac{\partial \ln \Sigma_g}{\partial R} +
% \frac{\partial}{\partial R} \left(
% \frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R} +
% \frac{\partial \ln v_{r,g}}{\partial R}
% \right) \right]
% \\
% &= -\tau_\star v_{r,g} \left[
% \frac{1 - N}{R} \frac{\partial \ln \Sigma_g}{\partial R} +
% (1 - N) \left(\frac{\partial \ln \Sigma_g}{\partial R}\right)^2 +
% \left(\frac{-1}{R^2} + \frac{\partial^2 \ln \Sigma_g}{\partial R^2}\right)
% \right]
% \\
% &= -\tau_\star v_{r,g} \left[
% \frac{1 - N}{R} \frac{\partial \ln \Sigma_g}{\partial R} +
% (1 - N) \left(\frac{\partial \ln \Sigma_g}{\partial R}\right)^2 -
% \frac{1}{R^2}
% \right]
% \\
% &= \tau_\star v_{r,g} \left[
% \frac{N - 1}{R R_g} +
% \frac{N - 1}{R_g^2} +
% \frac{1}{R^2}
% \right],
\end{align}
$$

where in the last line we have substituted in an exponential surface density profile in the ISM with a scale radius of $R_g$ (i.e., $\partial \ln \Sigma_g / \partial R = -1 / R_g$).
We have also assumed that this surface density gradient does not have a significant second-derivative (i.e., that it is a pure exponential).
In the Milky Way, $R_g \approx 3.75$ kpc (Kalberla & Kerp 2009).

We then arrive at the following expression for the equilibrium gradient in the limit of a flat velocity profile:

$$
\begin{align}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} &= \left[
\tau_\star v_{r,g} \left(
\frac{1 - N}{R R_g} -
\frac{1 - N}{R_g^2} +
\frac{1}{R^2} \right) +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{N - 1}{R_g} -
\frac{1}{R_\text{sfh}}
\right) \right] \left(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\tau_\star v_{r,g} \left(
\frac{1}{R} +
\frac{N - 2}{R_g}
\right)
\right)^{-1}
\end{align}
$$

In [24]:
N = 1.5 # K-S power-law index
Rg = 3.75 # kpc
vgas = -0.5 # km/s ~ kpc/Gyr
taustar = 2 * m.exp((N - 1) * 8 / Rg) # Gyr
R = 8 # kpc
recycling = 0.4
tausfh = 13 # Gyr
Rsfh = 4.7 # kpc
z_on_zeq = 10**-0.05

# numerator = taustar * vgas * ((1 - N) / (R * Rg) - (1 - N) / Rg**2 + 1 / R**2)
denominator = 1 - recycling - taustar / (N * tausfh)
# denominator -= taustar * vgas / Rg
denominator += taustar * vgas * (1 / R + (N - 2) / Rg)
# denominator += taustar * vgas * (1 / R + (N - 2) / Rg)
numerator = taustar / (N * tausfh) * ((N - 1) / Rg - 1 / Rsfh)
# numerator += taustar * vgas / Rg**2 * (N - 1)
numerator += taustar * vgas * ((1 - N) / (R * Rg) - (1 - N) / Rg**2 + 1 / R**2)
dlnzeqdr = numerator / denominator
print(numerator)
print(denominator)
dlnzeqdr = numerator / denominator
print(dlnzeqdr)
print(dlnzeqdr / m.log(10))
gradeq = z_on_zeq * dlnzeqdr + (1 - z_on_zeq) * (1 - N) / Rg
gradeq /= m.log(10)
print(gradeq)

-0.12395860898105548
0.32619575077458063
-0.3800129483192372
-0.16503752650683032
-0.15338706599057472


In [11]:
numerator = taustar / (N * tausfh) * ((N - 1) / Rg - 1 / Rsfh)
numerator += taustar * vgas / Rg**2 * (N - 1)
denominator = 1 - recycling - taustar / (N * tausfh) - taustar * vgas / Rg
dlnzeqdr = numerator / denominator
print(dlnzeqdr / m.log(10))
gradeq = z_on_zeq * dlnzeqdr + (1 - z_on_zeq) * (1 - N) / Rg
gradeq /= m.log(10)
print(gradeq)

-0.05121428845978253
-0.05194199829972272


Potential Well Deepening
--

In the potential well deepening scenario, the radial velocity is given by the growth in stellar mass:

$$
v_{r,g} = -R \gamma \frac{\partial \ln M_\star}{\partial t},
$$

where $\gamma \approx 0.2$ is the observed power-law index of the relationship between circular velocity and stellar mass. The velocity gradient then follows as

$$
\frac{\partial \ln v_{r,g}}{\partial R} = \frac{1}{R}.
$$

This velocity profile results in the following expression for $\mu_g$ and its radial derivative:

$$
\begin{align}
\mu_g &\rightarrow -\tau_\star v_{r,g} \left(
\frac{2}{R} +
\frac{\partial \ln \Sigma_g}{\partial R}
\right)
\\
\implies \frac{\partial \mu_g}{\partial R}
&= \mu_g \left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right) -
\tau_\star v_{r,g} \frac{\partial}{\partial R} \left(
\frac{2}{R} + \frac{\partial \ln \Sigma_g}{\partial R}
\right)
\\
&\approx -\tau_\star v_{r,g} \left(
(1 - N) \frac{\partial \ln \Sigma_g}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right) -
\frac{2 \tau_\star v_{r,g}}{R^2}
\\
&= \tau_\star v_{r,g} \left(
\frac{1 - N}{R_g} - \frac{1}{R}
\right) -
\frac{2 \tau_\star v_{r,g}}{R^2}.
\end{align}
$$

Plugging this derivative into the expression for $\nabla \ln Z_{\alpha,\text{eq}}$:

$$
\begin{align}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} &= \left[
\tau_\star v_{r,g} \left(
\frac{1 - N}{R_g} - \frac{1}{R}
\right) -
\frac{2 \tau_\star v_{r,g}}{R^2} +
\frac{\tau_\star}{N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\text{sfh}}{\partial R} -
\frac{1 - N}{R_g}
\right)
\right] \left(
1 - r + \frac{\tau_\star}{N \tau_\text{sfh}} +
\tau_\star v_{r,g} \left(
\frac{3}{R} + \frac{N - 2}{R_g}
\right)
\right)^{-1}
\end{align}
$$

Angular Momentum Dilution
--

In the angular momentum dilution scenario, the radial gas velocity is set by the angular momentum that the ISM exchanges with inflows and outfows:

$$
v_{r,g} = R\left[
\frac{\dot \Sigma_\text{out}}{\Sigma_g} \left(1 - \beta_{\phi,\text{out}}\right) -
\frac{\dot \Sigma_\text{in}}{\Sigma_g} \left(1 - \beta_{\phi,\text{in}}\right)
\right],
$$

where $\beta_{\phi,\text{out}}$ and $\beta_{\phi,\text{in}}$ are the ratio of specific angular momenta of the infalling and accretion material to the ISM, respectively (i.e., $\beta_{\phi,\text{in}} \equiv v_{\phi,\text{in}} / v_{\phi,g}$). Substituting in $\Sigma_g = \dot \Sigma_\star \tau_\star$, the ratio of accretion per unit star formation, and the mass-loading factor:

$$
\begin{align}
v_{r,g} &= R\left[
\frac{\eta}{\tau_\star} \left(1 - \beta_{\phi,\text{out}}\right) -
\left(\frac{1 + \eta - \mu_g - r}{\tau_\star} +
\frac{\dot \Sigma_g}{\Sigma_g} \right)
\left(1 - \beta_{\phi,\text{in}}\right)
\right]
\end{align}
$$

Finding an expression for the velocity gradient $\partial \ln v_{r,g} / \partial R$ is trivial if one realizes that this term exists inside of $\mu_g$.
One can also solve for $\mu_g$ algebraically given the above expression, and differentiate with radius from there.
Following this route:

$$
\begin{align}
1 + \eta - \mu_g - r - \frac{\tau_\star}{N \tau_\text{sfh}}
&= \eta \left(\frac{1 - \beta_{\phi,\text{out}}}{1 - \beta_{\phi,\text{in}}}\right) -
\frac{\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})}
\\
\implies \mu_g
&= 1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\eta \left(
\frac{\beta_{\phi,\text{out}} - \beta_{\phi,\text{in}}}{1 - \beta_{\phi,\text{in}}}
\right) +
\frac{\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})}
\\
&\rightarrow 1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\frac{\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})},
% \\
% \implies \frac{\partial \mu_g}{\partial R}
% &= \frac{-\tau_\star}{N \tau_\text{sfh}} \left(
% \frac{\partial \ln \tau_\star}{\partial R} -
% \frac{\partial \ln \tau_\text{sfh}}{\partial R}
% \right) +
% \frac{\partial \eta}{\partial R} \left(
% \frac{\beta_{\phi,\text{out}} - \beta_{\phi,\text{in}}}{1 - \beta_{\phi,\text{in}}}
% \right)
\end{align}
$$

where we have applied the $\eta \rightarrow 0$ limit in the last line.

From here, we can substitute in the definition of $\mu_g$ and isolate the velocity gradient:

$$
\begin{align}
-\tau_\star v_{r,g} \left(
\frac{1}{R} +
\frac{\partial \ln \Sigma_g}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)
&= 1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\frac{\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})}
\\
\implies \frac{1}{R} +
\frac{\partial \ln \Sigma_g}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
&= \frac{-1}{R (1 - \beta{\phi,\text{in}})} -
\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{\tau_\star v_{r,g}}
\\
\implies \frac{\partial \ln v_{r,g}}{\partial R}
&= \frac{-1}{R (1 - \beta{\phi,\text{in}})} -
\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{\tau_\star v_{r,g}} -
\frac{\partial \ln \Sigma_g}{\partial R} -
\frac{1}{R}
\end{align}
$$

And now differentiating $\mu_g$ with respect to radius, starting with our expression above:

$$
\begin{align}
\frac{\partial \mu_g}{\partial R}
&= \frac{-\tau_\star}{N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right) +
\frac{\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})} \left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial r} -
\frac{1}{R}
\right)
\\
&= \frac{-\tau_\star}{N \tau_\text{sfh}} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{\partial \ln \tau_\text{sfh}}{\partial R}
\right) +
\frac{\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})} \left(
\frac{\partial \ln \tau_\star}{\partial R} -
\frac{1}{R (1 - \beta{\phi,\text{in}})} -
\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{\tau_\star v_{r,g}} -
\frac{\partial \ln \Sigma_g}{\partial R} -
\frac{2}{R}
\right)
\end{align}
$$

Plugging the expressions for $\partial \mu_g / \partial R$ and $\partial \ln v_{r,g} / \partial R$ into our expression for $\partial \ln Z_{\alpha,\text{eq}} / \partial R$, along with $\partial \ln \tau_\star / \partial R = (1 - N) \partial \ln \Sigma_g / \partial R$:

$$
\begin{align}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}
&= \left[
\frac{\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})} \left(
-N \frac{\partial \ln \Sigma_g}{\partial R} -
\frac{1}{R (1 - \beta{\phi,\text{in}})} -
\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{\tau_\star v_{r,g}} -
\frac{2}{R}
\right)
\right] \left(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\tau_\star v_{r,g} \left(
\frac{1}{R} +
(2 - N)\frac{\partial \ln \Sigma_g}{\partial R} +
2\frac{\partial \ln v_{r,g}}{\partial R}
\right)
\right)^{-1}
\\
&= \left[
\frac{\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})} \left(
-N \frac{\partial \ln \Sigma_g}{\partial R} -
\frac{1}{R (1 - \beta{\phi,\text{in}})} -
\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{\tau_\star v_{r,g}} -
\frac{2}{R}
\right)
\right] \Bigg(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\tau_\star v_{r,g} \Bigg(
\frac{1}{R} +
(2 - N)\frac{\partial \ln \Sigma_g}{\partial R}
\\
& - \frac{2}{R (1 - \beta_{\phi,\text{in}})} -
2\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{\tau_\star v_{r,g}} -
2\frac{\partial \ln \Sigma_g}{\partial R} -
\frac{2}{R}
\Bigg)
\Bigg)^{-1}
\\
&= \left[
\frac{\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})} \left(
N \frac{\partial \ln \Sigma_g}{\partial R} +
\frac{1}{R (1 - \beta{\phi,\text{in}})} +
\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{\tau_\star v_{r,g}} +
\frac{2}{R}
\right)
\right] \left(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\tau_\star v_{r,g} \left(
N \frac{\partial \ln \Sigma_g}{\partial R} +
\frac{2}{R (1 - \beta_{\phi,\text{in}})} +
\frac{1}{R}
\right)
\right)^{-1}
\\
&= \left[
\frac{1 - r - \tau_\star / N \tau_\text{sfh}}{R (1 - \beta_{\phi,\text{in}})} +
\frac{\tau_\star v_{r,g}}{R (1 - \beta_{\phi,\text{in}})} \left(
N \frac{\partial \ln \Sigma_g}{\partial R} +
\frac{1}{R (1 - \beta{\phi,\text{in}})} +
\frac{2}{R}
\right)
\right] \left(
1 - r - \frac{\tau_\star}{N \tau_\text{sfh}} +
\tau_\star v_{r,g} \left(
N \frac{\partial \ln \Sigma_g}{\partial R} +
\frac{2}{R (1 - \beta_{\phi,\text{in}})} +
\frac{1}{R}
\right)
\right)^{-1}
\end{align}
$$

The River Limit
--

In the rivir limit, the formula we derived for $\partial \ln Z_{\alpha,\text{eq}} / \partial R$ above breaks down; the expression reaches a $0 / 0$ indeterminate form.
This result is unsurprising considering that the river limit is an edge case by definition.

The easiest way to see why this is the case and what the solution to the equilibrium gradient should be is to remember the following alternative form for the local equilibrium abundance:

$$
Z_{\alpha,\text{eq}} = \frac{y_\alpha}{
\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} + \tau_\star v_{r,g} \frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R}}.
$$

By definition, $\dot \Sigma_\text{in} \rightarrow 0$ in the river limit, which results in the equilibrium abundance being undefined. Only its radial gradient is defined:

$$
\frac{\partial Z_{\alpha,\text{eq}}}{\partial R} = \frac{y_\alpha}{\tau_\star v_{r,g}}.
$$

However, we can still solve for the slope if we know how $\tau_\star$ and $v_{r,g}$ scale with radius.
Our prescription for $\tau_\star$ results in an exponential increase radius with a scale length of $(N - 1) / R_g$.
The velocity profile follows from our solution to $\mu_g$:

$$
\begin{align}
\mu_g &\rightarrow 1 + \eta - r - \frac{\tau_\star}{N \tau_\text{sfh}}
\\
\implies \frac{\partial v_{r,g}}{\partial R} +
v_{r,g} \left(
\frac{1}{R} +
\frac{\partial \ln \Sigma_g}{\partial R}
\right)
&= \frac{-\dot \Sigma_g}{\Sigma_g} -
\frac{1 + \eta - r}{\tau_\star}
\end{align}
$$

This expression is a linear ODE whose solution is known. Also applying the limit that $\eta = 0$:

$$
\begin{align}
v_{r,g} &= \exp\left(
-\int \left(\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R}\right) dR
\right) \left[
\int \exp \left(
\int \left(\frac{1}{R} + \frac{\partial \ln \Sigma_g}{\partial R}\right) dR
\right) \left(
\frac{-\dot \Sigma_g}{\Sigma_g} -
\frac{1 - r}{\tau_\star}
\right) dR + C
\right]
\\
&= \exp \left(-\ln R + \frac{R}{R_g}\right) \left[
\int \exp \left(\ln R - \frac{R}{R_g}\right) \left(
\frac{-\dot \Sigma_g}{\Sigma_g} -
\frac{1 - r}{\tau_\star}
\right) dR + C
\right]
\\
&= \frac{e^{R / R_g}}{R} \left[
\int Re^{-R / R_g} \left(
\frac{-\dot \Sigma_g}{\Sigma_g} -
\frac{1 - r}{\tau_\star}
\right) dR + C
\right]
\\
&= \frac{-e^{R / R_g}}{R} \left[
\int Re^{-R / R_g} \left(
\frac{\dot \Sigma_g}{\Sigma_g} +
\frac{1 - r}{\tau_\star}
\right) dR + C
\right]
\\
&= \frac{-e^{R / R_g}}{R} \left[
\int \left(
R e^{-R / R_g} \left(\frac{1 - r}{\tau_\star} -
\frac{1}{N \tau_\text{sfh}}\right)\right)
dR + C
\right]
\\
&= \frac{-e^{R / R_g}}{R} \left[
\int \left(
R e^{-R / R_g} \left(\frac{1 - r}{\tau_{\star,0}} e^{-(N - 1)R / R_g} -
\frac{1}{N \tau_\text{sfh,0}} e^{-R / R_\text{sfh}}\right)\right)
dR + C
\right]
\\
&= \frac{-e^{R / R_g}}{R} \left[
\int \left(
\frac{1 - r}{\tau_{\star,0}} Re^{-N R / R_g} -
\frac{R}{N \tau_\text{sfh,0}} e^{-R (1 / R_g + 1 / R_\text{sfh})}\right)
dR + C
\right]
\\
&= \frac{-e^{R / R_g}}{R} \left[
\frac{1 - r}{\tau_{\star,0}} e^{- N R / R_g} \left(
\frac{-R R_g}{N} - 
\left(\frac{R_g}{N}\right)^2
\right) -
\frac{1}{N \tau_\text{sfh,0}} e^{-R (1 / R_g + 1 / R_\text{sfh})} \left(
\frac{-R R_\text{sfh} R_g}{R_\text{sfh} + R_g} -
\left(\frac{R_\text{sfh} R_g}{R_\text{sfh} + R_g}\right)^2
\right) +
C
\right]
\\
&=
\frac{1 - r}{\tau_{\star,0}} e^{(1 - N) R / R_g} \left(
\frac{R_g}{N} + \frac{R_g^2}{R N^2}
\right) -
\frac{1}{N \tau_\text{sfh,0}} e^{-R / R_\text{sfh}} \left(
\frac{R_\text{sfh} R_g}{R_\text{sfh} + R_g} +
\frac{R_\text{sfh}^2 R_g^2}{R \left(R_\text{sfh} + R_g\right)^2}
\right) +
C \frac{e^{R / R_g}}{R}
% \\
% &= \frac{1}{N \tau_\text{sfh,0}} e^{-R / R_\text{sfh}} \left(
% \frac{R_\text{sfh} R_g}{R_\text{sfh} - R_g} -
% \frac{1}{R}
% \left(\frac{R_\text{sfh} R_g}{R_\text{sfh} - R_g}\right)^2
% \right) -
% \frac{1 - r}{\tau_{\star,0}} e^{(1 - N)R / R_g} \left(
% \frac{R_g}{2 - N} -
% \frac{1}{R} \left(\frac{R_g}{2 - N}\right)^2
% \right) +
% C\frac{e^{R / R_g}}{R}
\end{align}
$$

In the above expression, the first two terms decay with radius while the final one grows.
Therefore, at sufficiently large radius, the velocity scales as

$$
v_{r,g} \approx C \frac{e^{R / R_g}}{R},
$$

and therefore a simplified approximation of the velocity gradient:

$$
\begin{align}
\frac{\partial v_{r,g}}{\partial R} &\approx C \left(
\frac{e^{R / R_g}}{R R_g} - \frac{e^{R / R_g}}{R^2}
\right)
\\
&\approx C \frac{e^{R / R_g}}{R} \left(
\frac{1}{R_g} - \frac{1}{R}
\right)
\\
\implies \frac{\partial \ln v_{r,g}}{\partial R}
&\approx \frac{1}{R_g} - \frac{1}{R}
\\
&\approx \frac{1}{R_g}.
\end{align}
$$

A mathematical description of the velocity profile in the river limit is complex at small radii, but approaches an exponential profile with a scale length of $R_g$. Therefore,

$$
\frac{\partial \ln v_{r,g}}{\partial R} \approx \frac{-\partial \ln \Sigma_g}{\partial R}.
$$

Since the river scenario is a theoretical limit at which accretion cannot physically happen at any larger radius and leads to metallicity gradients in tension with the observations, we can argue as a conclusion of this result that

$$
\frac{\partial \ln v_{r,g}}{\partial R} \ll \frac{-\partial \ln \Sigma_g}{\partial R}
$$

in the Milky Way.

The solution to the above expression for $\nabla \ln Z_{\alpha,\text{eq}}$ is a quadratic.

$$
\begin{align}
\frac{\partial \ln Z_{\alpha,\text{eq}}}{\partial R} &=
\frac{1}{2 \tau_\star v_{r,g}}
\left[
-\left(
\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} +
\tau_\star v_{r,g} 
\left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)
\right)
\pm 
\sqrt{
\left( 
\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} +
\tau_\star v_{r,g} 
\left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)
\right)^2 +
4 \tau_\star v_{r,g} \left(
\frac{\partial \mu_g}{\partial R} +
\frac{\tau_\star}{N \tau_\text{sfh}}
\left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right) -
\frac{\partial \eta}{\partial R}
\right)
}
\right]
\\
&= \frac{1}{2 \tau_\star v_{r,g}}
\left[
-\left(
\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} +
\tau_\star v_{r,g} 
\left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)
\right)
\pm 
\sqrt{
\left(\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star}\right)^2 +
2\frac{\dot \Sigma_\text{in}}{\dot \Sigma_\star} \tau_\star v_{r,g} \left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right) +
\left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right)^2 +
4 \tau_\star v_{r,g} \left(
\frac{\partial \mu_g}{\partial R} +
\frac{\tau_\star}{N \tau_\text{sfh}}
\left(
\frac{\partial \ln \tau_\star}{\partial R} +
\frac{\partial \ln v_{r,g}}{\partial R}
\right) -
\frac{\partial \eta}{\partial R}
\right)
}
\right]
\end{align}
$$